In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory

In [2]:
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")

# memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input_text",
    output_key="processed_text"
)

### 1-Detect language:

In [3]:
detect_prompt = ChatPromptTemplate.from_template(
    "Detect the language of the following text:\n\n{input_text}"
)
detect_chain = LLMChain(llm=llm, prompt=detect_prompt, output_key="language")

### 2-Translate text into english:

In [4]:
translate_prompt = ChatPromptTemplate.from_template(
    "Translate the following text into English:\n\n{input_text}"
)
translate_chain = LLMChain(llm=llm, prompt=translate_prompt, output_key="english_text")

### 3-Summerize or Paraphrase:

In [21]:
process_prompt = ChatPromptTemplate.from_template(
    "You need to {mode} the following English text into a concise summary (max 2-3 sentences):\n\n{english_text}"
)
process_chain = LLMChain(llm=llm, prompt=process_prompt, output_key="processed_text")

### SequentialChain:

In [6]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    memory=memory,
    chains=[detect_chain, translate_chain, process_chain],
    input_variables=["input_text", "mode"],
    output_variables=["language", "english_text", "processed_text"],
    verbose=True
)

### Simple Use (summerize):

In [7]:
text1 = """Le produit est arrivé très rapidement, ce qui était une bonne surprise. 
Cependant, l’emballage était endommagé et cela a légèrement abîmé l’article. 
Dans l’ensemble, je suis satisfait de la qualité du produit, mais je pense que 
l’entreprise devrait faire plus attention à la protection des colis."""

In [8]:
result1 = overall_chain({
    "input_text": text1,
    "mode": "summarize"
})



> Entering new SequentialChain chain...

> Finished chain.


In [9]:
print(result1)

{'input_text': 'Le produit est arrivé très rapidement, ce qui était une bonne surprise. \nCependant, l’emballage était endommagé et cela a légèrement abîmé l’article. \nDans l’ensemble, je suis satisfait de la qualité du produit, mais je pense que \nl’entreprise devrait faire plus attention à la protection des colis.', 'mode': 'summarize', 'chat_history': '', 'language': 'French', 'english_text': 'The product arrived very quickly, which was a pleasant surprise. However, the packaging was damaged and this slightly damaged the item. Overall, I am satisfied with the quality of the product, but I think the company should pay more attention to protecting packages.', 'processed_text': 'The product arrived quickly but the packaging was damaged, causing slight damage to the item. Overall, the quality of the product was satisfactory, but the company should improve its packaging to prevent damage in transit.'}


In [10]:
print("\n**************\n".join(f"{key}: {value}" for key, value in result1.items()))

input_text: Le produit est arrivé très rapidement, ce qui était une bonne surprise. 
Cependant, l’emballage était endommagé et cela a légèrement abîmé l’article. 
Dans l’ensemble, je suis satisfait de la qualité du produit, mais je pense que 
l’entreprise devrait faire plus attention à la protection des colis.
**************
mode: summarize
**************
chat_history: 
**************
language: French
**************
english_text: The product arrived very quickly, which was a pleasant surprise. However, the packaging was damaged and this slightly damaged the item. Overall, I am satisfied with the quality of the product, but I think the company should pay more attention to protecting packages.
**************
processed_text: The product arrived quickly but the packaging was damaged, causing slight damage to the item. Overall, the quality of the product was satisfactory, but the company should improve its packaging to prevent damage in transit.


In [11]:
text2 = """Heureusement, le service client a été très réactif et m’a proposé un remboursement 
partiel rapidement. Leur professionnalisme m’a convaincu de continuer à acheter chez eux."""

In [12]:
result2 = overall_chain({
    "input_text": text2,
    "mode": "summarize"
})



> Entering new SequentialChain chain...

> Finished chain.


In [13]:
print("\n**************\n".join(f"{key}: {value}" for key, value in result2.items()))

input_text: Heureusement, le service client a été très réactif et m’a proposé un remboursement 
partiel rapidement. Leur professionnalisme m’a convaincu de continuer à acheter chez eux.
**************
mode: summarize
**************
chat_history: Human: Le produit est arrivé très rapidement, ce qui était une bonne surprise. 
Cependant, l’emballage était endommagé et cela a légèrement abîmé l’article. 
Dans l’ensemble, je suis satisfait de la qualité du produit, mais je pense que 
l’entreprise devrait faire plus attention à la protection des colis.
AI: The product arrived quickly but the packaging was damaged, causing slight damage to the item. Overall, the quality of the product was satisfactory, but the company should improve its packaging to prevent damage in transit.
**************
language: French
**************
english_text: Fortunately, the customer service was very responsive and offered me a partial refund quickly. Their professionalism convinced me to continue buying from them.

In [14]:
print("\n=== Dialogue History ===")
for msg in memory.chat_memory.messages:
    print(msg.content)


=== Dialogue History ===
Le produit est arrivé très rapidement, ce qui était une bonne surprise. 
Cependant, l’emballage était endommagé et cela a légèrement abîmé l’article. 
Dans l’ensemble, je suis satisfait de la qualité du produit, mais je pense que 
l’entreprise devrait faire plus attention à la protection des colis.
The product arrived quickly but the packaging was damaged, causing slight damage to the item. Overall, the quality of the product was satisfactory, but the company should improve its packaging to prevent damage in transit.
Heureusement, le service client a été très réactif et m’a proposé un remboursement 
partiel rapidement. Leur professionnalisme m’a convaincu de continuer à acheter chez eux.
The customer service was responsive and quickly offered a partial refund, demonstrating professionalism that convinced the individual to continue buying from them.


In [15]:
final_text1 = result1["processed_text"]
final_text2 = result2["processed_text"]

concatenated_text = final_text1 + " " + final_text2

print(concatenated_text)

The product arrived quickly but the packaging was damaged, causing slight damage to the item. Overall, the quality of the product was satisfactory, but the company should improve its packaging to prevent damage in transit. The customer service was responsive and quickly offered a partial refund, demonstrating professionalism that convinced the individual to continue buying from them.


## Final use: (summarize)

In [29]:
def process_two_texts():
    mode = input("Choose mode (summarize/paraphrase): ").strip().lower()
    
    text1 = input("Enter the first text:\n").strip()
    text2 = input("Enter the second text:\n").strip()
    
    result1 = overall_chain({"input_text": text1, "mode": mode})
    print("\n=== Text 1 translated into English ===")
    print(result1["english_text"])
    
    result2 = overall_chain({"input_text": text2, "mode": mode})
    print("\n=== Text 2 translated into English ===")
    print(result2["english_text"])
    
    # Final result
    final_result = result1["processed_text"] + " " + result2["processed_text"]
    
    # Automate final title based on chosen mode
    mode_title = "summarized" if mode == "summarize" else "paraphrased"
    print(f"\n=== Concatenated final text ({mode_title}) ===")
    print(final_result)

In [30]:
process_two_texts()

Choose mode (summarize/paraphrase): summarize
Enter the first text:
Le produit que j'ai commandé est arrivé rapidement et en bon état général.  Cependant, l'emballage extérieur présentait quelques déchirures qui ont légèrement abîmé le contenu.  Malgré cela, la qualité du produit reste satisfaisante et conforme à la description.  Je recommande néanmoins de renforcer l'emballage pour éviter ce genre de désagrément.
Enter the second text:
Le service client a été très réactif et m'a aidé à résoudre un problème lié à ma commande.  Ils ont répondu à toutes mes questions avec patience et professionnalisme.  Grâce à leur efficacité, j'ai pu obtenir un remboursement partiel rapidement.  Cette expérience positive me donne confiance pour continuer à acheter sur ce site.


> Entering new SequentialChain chain...

> Finished chain.

=== Text 1 translated into English ===
The product I ordered arrived quickly and in good overall condition. However, the outer packaging had some tears that slightly d

## Final use: (paraphrase)

In [31]:
process_two_texts()

Choose mode (summarize/paraphrase): paraphrase
Enter the first text:
Le produit que j'ai commandé est arrivé rapidement et en bon état général.  Cependant, l'emballage extérieur présentait quelques déchirures qui ont légèrement abîmé le contenu.  Malgré cela, la qualité du produit reste satisfaisante et conforme à la description.  Je recommande néanmoins de renforcer l'emballage pour éviter ce genre de désagrément.
Enter the second text:
Le service client a été très réactif et m'a aidé à résoudre un problème lié à ma commande.  Ils ont répondu à toutes mes questions avec patience et professionnalisme.  Grâce à leur efficacité, j'ai pu obtenir un remboursement partiel rapidement.  Cette expérience positive me donne confiance pour continuer à acheter sur ce site.


> Entering new SequentialChain chain...

> Finished chain.

=== Text 1 translated into English ===
The product I ordered arrived quickly and in good overall condition. However, the outer packaging had some tears that slightly 